In [ ]:
"""Compare differences in time delay changes between medication and stimulation."""

import os
import sys
from pathlib import Path
import numpy as np
import copy
import pandas as pd
from matplotlib import pyplot as plt
import pte_decode

cd_path = Path(os.getcwd()).absolute().parent
sys.path.append(os.path.join(cd_path, "coherence"))

import matplotlib
matplotlib.rc('xtick', labelsize=6)
matplotlib.rc('ytick', labelsize=6)
matplotlib.rc('legend', fontsize=6)
matplotlib.rc("font", size=6, family="Arial")
matplotlib.rc('axes', labelsize=7)
matplotlib.rc('axes', titlesize=7)
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

prop_cycle = plt.rcParams["axes.prop_cycle"]
def_colors = prop_cycle.by_key()["color"]

n_perm = 100_000
two_sided = False
alpha = 0.05

FOLDERPATH_ANALYSIS = (
    "\\\\?\\C:\\Users\\tsbin\\OneDrive - Charité - "
    "Universitätsmedizin Berlin\\PROJECT ECOG-LFP Coherence\\Analysis\\"
    "Analysis\\Results\\BIDS_01_Berlin_Neurophys\\sub-multi\\ses-multi"
)

subregions_mapping = {
    "Motor cortex": "motor",
    "Sensory cortex": "sensory",
    "Parietal cortex": "parietal",
    "Prefrontal cortex": "prefrontal"
}

In [ ]:
# FIND SHARED SUBJECTS FOR MED & STIM
data_med = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_con_tde-MedOffOn_multi_sub.pkl")
)
med_subs = np.unique(data_med["sub"])

data_stim = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_con_tde-StimOffOn_multi_sub.pkl")
)
stim_subs = np.unique(data_stim["sub"])

shared_subs = list(set(med_subs).intersection(stim_subs))

In [ ]:
# COMPUTE CHANGES IN CORTEX -> STN TIME DELAY WITH MED & STIM
times = copy.deepcopy(data_med["timepoints"][0])
data_med["timepoints"] = [times] * len(data_med["seed_types"])
data_med = pd.DataFrame.from_dict(data_med)
data_stim = pd.DataFrame.from_dict(data_stim)

subregion_i = 0
tde_med_diff = {}
tde_stim_diff = {}
for subregion_name, subregion_label in subregions_mapping.items():
    tde_med_diff[subregion_label] = []
    tde_stim_diff[subregion_label] = []
    for sub in shared_subs:
        # MED TDE
        sub_off_idcs = ((data_med["med"] == "Off") & (data_med["freq_band_names"] == "all")
                        & (data_med["seed_subregions"] == subregion_label)
                        & (data_med["sub"] == sub)
                        & ~((data_med["tde-i_standard_tau_ci_80_low"] < 0) & (data_med["tde-i_standard_tau_ci_80_high"] > 0)))
        tde_med_off_sub = np.array(data_med["tde-i_standard"][sub_off_idcs].to_list())
        if tde_med_off_sub.size > 0:
            tde_med_off_sub = tde_med_off_sub.mean(0)

        sub_on_idcs = ((data_med["med"] == "On") & (data_med["freq_band_names"] == "all")
                        & (data_med["seed_subregions"] == subregion_label)
                        & (data_med["sub"] == sub)
                        & ~((data_med["tde-i_standard_tau_ci_80_low"] < 0) & (data_med["tde-i_standard_tau_ci_80_high"] > 0)))
        tde_med_on_sub = np.array(data_med["tde-i_standard"][sub_on_idcs].to_list())
        if tde_med_on_sub.size > 0:
            tde_med_on_sub = tde_med_on_sub.mean(0)

        # STIM TDE
        sub_off_idcs = ((data_stim["stim"] == "Off") & (data_stim["freq_band_names"] == "all")
                        & (data_stim["seed_subregions"] == subregion_label)
                        & (data_stim["sub"] == sub)
                        & ~((data_stim["tde-i_standard_tau_ci_80_low"] < 0) & (data_stim["tde-i_standard_tau_ci_80_high"] > 0)))
        tde_stim_off_sub = np.array(data_stim["tde-i_standard"][sub_off_idcs].to_list())
        if tde_stim_off_sub.size > 0:
            tde_stim_off_sub = tde_stim_off_sub.mean(0)

        sub_on_idcs = ((data_stim["stim"] == "On") & (data_stim["freq_band_names"] == "all")
                        & (data_stim["seed_subregions"] == subregion_label)
                        & (data_stim["sub"] == sub)
                        & ~((data_stim["tde-i_standard_tau_ci_80_low"] < 0) & (data_stim["tde-i_standard_tau_ci_80_high"] > 0)))
        tde_stim_on_sub = np.array(data_stim["tde-i_standard"][sub_on_idcs].to_list())
        if tde_stim_on_sub.size > 0:
            tde_stim_on_sub = tde_stim_on_sub.mean(0)

        # DIFFERENCES (OFF - ON)
        if tde_med_off_sub.size > 0 and tde_med_on_sub.size > 0 and tde_stim_off_sub.size > 0 and tde_stim_on_sub.size > 0:
            tde_med_diff[subregion_label].append(tde_med_off_sub - tde_med_on_sub)
            tde_stim_diff[subregion_label].append(tde_stim_off_sub - tde_stim_on_sub)
    
    tde_med_diff[subregion_label] = np.array(tde_med_diff[subregion_label])
    tde_stim_diff[subregion_label] = np.array(tde_stim_diff[subregion_label])
    
    subregion_i += 1

In [ ]:
# FIND WHETHER THERE IS A SIGNIFICANT DIFFERENCE BETWEEN MED & STIM EFFECTS (FOR MOTOR CORTEX -> STN)
ltime = 0
htime = 51
fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=tde_med_diff["motor"][:, times.index(ltime):times.index(htime)+1].T,
    x_2=tde_stim_diff["motor"][:, times.index(ltime):times.index(htime)+1].T,
    times=np.array(times[times.index(ltime):times.index(htime)+1]),
    data_labels=["Med", "Stim"],
    x_label="Time (ms)",
    y_label="Est (A.U.)",
    two_tailed=True,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="Cortex -> STN",
    colour=["#376EB4", "#71BCAD"]
)